 **Refer the below link for more details**: https://huggingface.co/docs/transformers/en/tasks/summarization

In [17]:
!nvidia-smi

Sun Jun 16 14:37:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.30                 Driver Version: 546.30       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P8               6W /  40W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.1.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
alembic                       1.13.1
anaconda-client               1.9.0
anaconda-navigator            2.1.4
anaconda-project              0.10.2
annotated-types               0.6.0
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
back

pyarrow-hotfix                0.6
pyasn1                        0.4.8
pyasn1-modules                0.2.8
pybcj                         1.0.2
pycodestyle                   2.7.0
pycosat                       0.6.3
pycparser                     2.21
pycryptodomex                 3.20.0
pyct                          0.4.6
pycurl                        7.44.1
pydantic                      2.7.1
pydantic_core                 2.18.2
PyDispatcher                  2.0.5
pydocstyle                    6.1.1
pyerfa                        2.0.0
pyflakes                      2.3.1
Pygments                      2.11.2
PyHamcrest                    2.0.2
PyJWT                         2.1.0
pylint                        2.9.6
pyls-spyder                   0.4.0
PyNaCl                        1.4.0
pyodbc                        4.0.32
pyOpenSSL                     21.0.0
pyparsing                     3.0.4
pyppmd                        1.1.0
pyreadline                    2.1
pyrsistent                 

-  **Sacrebleu** - is a metric commonly used for evaluating the quality of machine-generated translations, typically in the context of machine translation tasks. it is often used in NLP Competetions and research to assess the performance of translation models. 
-  **Rouge(Recall-Oriented understudy for gisting evaluation)** - is a set of metrics used for evaluating the quality of summaries by comparing them to reference summaries. Rouge metrics are commonly used in natural language processing tasks, particularly in the evalution of text summarization systems 
- **transformers[sentencepiece]** - to convert sentences into tokens

In [19]:
!pip install transformers[sentencepiece] dataset sacrebleu rouge_score py7zr -q

In [20]:
# To distribute the process between the cores inside GPU
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0
  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)


In [21]:
import torch
device = "gpu" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saipr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [45]:
from datasets import load_dataset, load_from_disk, load_metric

In [46]:
model = "google/pegasus-cnn_dailymail"

In [47]:
# loading the model

tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- Training of the model (finetuning)
- Inferencing - Loading the pretrained model and just predicting using the loaded model

In [48]:
dataset = load_dataset("samsum")

In [49]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [50]:
print(dataset["train"]["dialogue"][10])
print("\nSummary: \n")
print(dataset["train"]["summary"][10])

Lucas: Hey! How was your day?
Demi: Hey there! 
Demi: It was pretty fine, actually, thank you!
Demi: I just got promoted! :D
Lucas: Whoa! Great news!
Lucas: Congratulations!
Lucas: Such a success has to be celebrated.
Demi: I agree! :D
Demi: Tonight at Death & Co.?
Lucas: Sure!
Lucas: See you there at 10pm?
Demi: Yeah! See you there! :D

Summary: 

Demi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.


In [51]:
for split in dataset:
    print(split, ": ", "column_names: ", dataset[split].column_names, "length of the datasets: ", len(dataset[split]) )

train :  column_names:  ['id', 'dialogue', 'summary'] length of the datasets:  14732
test :  column_names:  ['id', 'dialogue', 'summary'] length of the datasets:  819
validation :  column_names:  ['id', 'dialogue', 'summary'] length of the datasets:  818


In [52]:
# Converting examples to features
def convert_examples_to_features(data_batch):
    
    input_encoding = tokenizer(data_batch["dialogue"], max_length=1024, truncation=True)
    target_encoding=  tokenizer(data_batch["summary"], max_length=128, truncation=True)
    
    return {
        'input_ids' : input_encoding['input_ids'],
        'attention_mask' : input_encoding['attention_mask'],
        'labels' : target_encoding['input_ids']
    }

In [53]:
dataset_en=dataset.map(convert_examples_to_features, batched=True)

In [54]:
dataset_en["train"]["input_ids"]

[[12195,
  151,
  125,
  7091,
  3659,
  107,
  842,
  119,
  245,
  181,
  152,
  10508,
  151,
  7435,
  147,
  12195,
  151,
  125,
  131,
  267,
  650,
  119,
  3469,
  29344,
  1],
 [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 [4776,
  151,
  4451,
  108,
  180,
  131,
  116,
  164,
  152,
  5377,
  151,
  6843,
  4301,
  83678,
  108,
  125,
  140,
  313,
  112,
  171,
  1425,
  113,
  1549,
  155,
  2371,
  164,
  64428,
  4776,
  151,
  463,
  368,
  119,
  511,
  124,
  557,
  152,
  5377,
  151,
  4384,
  119,
  235,
  108,
  18857,
  1549,
  111,
  1596,
  2073,
  56616,
  161,
  418,
  5377,
  151,
  3183,
  3469,
  125,
  131,
  267,
  696,
  161,
  130,
  116,
  111,
  171,
  579,
  5377,
  151,
  184,
  195,
  313,
  112,
  38244,
  114,
  5713,
  167,
  1088,
  113,
  1553,
  125,
  131,
  267,
  1461,
  181,
  38244,
  

In [55]:
dataset_en["train"]["attention_mask"][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### Training

In [56]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [57]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

C:\Users\saipr\anaconda3\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [58]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_en["test"],
                  eval_dataset=dataset_en["validation"])

In [59]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0706825677086327, metrics={'train_runtime': 3138.0387, 'train_samples_per_second': 0.261, 'train_steps_per_second': 0.016, 'total_flos': 313450454089728.0, 'train_loss': 3.0706825677086327, 'epoch': 0.9963369963369964})

#### Evalution

In [60]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [61]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

C:\Users\saipr\AppData\Local\Temp\ipykernel_22168\2696170338.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
C:\Users\saipr\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [71]:
score = calculate_metric_on_test_ds(
    dataset_en['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:52<00:00, 58.42s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.020386,0.0,0.020157,0.020553


In [65]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [66]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\spiece.model',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [68]:
#Load

tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [70]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_en["test"][0]["dialogue"]

reference = dataset_en["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
